In [56]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

pd.set_option("display.max_columns", 40)
pd.options.display.width=None
pd.options.display.float_format = '{:,.2f}'.format

In [57]:
#scikit-learn no tiene el error porcentual medio (MPE) lo vamos a crear nosotros
def mean_porcentual_error(yreal, ypred):
    return np.mean((yreal-ypred)/yreal)

# Explicación de las columnas

- **id:** Id de base de datos del registro
- **name:** Nombre de la publicación
- **operation_type:** Tipo de operación que se esta realizando con la propiedad
- **operation_currency:** Moneda de la operación
- **operation_amount:** Monto de la operación
- **expenses_currency:** Moneda de las expensas
- **expenses_amount:** Monto de las expensas
- **total_mts:** Total de metros de la propiedad
- **covered_mts:** Total de metros cubiertos de la propiedad
- **rooms:** Cantidad de ambientes
- **bedrooms:** Cantidad de dormitorios
- **bathrooms:** Cantidad de baños
- **garages:** Cantidad de garages
- **antique:** Antiguedad de la propiedad
- **building_layout:** Disposición del edificio
- **orientation:** Orientación de la propiedad
- **number_of_floors:** Cantidad de pisos del edificio
- **apartments_per_floor:** Departamentos por cada piso
- **real_estate_type:** Tipo de propiedad
- **posting_type:** Tipo de publicación
- **publisher_id:** Id de base de datos del publicador
- **publisher_name:** Nombre del publicador
- **address:** Dirección de la propiedad
- **city_id:** Id de base de datos del barrio/localidad de la propiedad
- **city:** Nombre del barrio de la propiedad
- **state_id:** Id de base de datos de la provincia de la propiedad
- **state:** Nombre de la provincia de la propiedad
- **country_id:** Id de base de datos del país de la propiedad
- **country:** Nombre del país de la propiedad
- **latitude:** Ubicación en latitud de la propiedad
- **longitude:** Ubicación en longitud de la propiedad
- **reserved:** Tiene reserva?
- **publication_antiquity:** Antiguedad de la publicación
- **url:** Dirección web de la publicación

- TODO: Agregar mas informacion sobre a que corresponde segun el codigo civil el real_estate_type
- TODO: Analizar por que terrenos tienen metros cubiertos
- TODO: En la eliminacion de datos por outlier (percentil 0.999), una idea mas copada seria calcular el percentil de cada variable agrupando antes por cantidad de ambientes por ejemplo.
- TODO: Ver si no conviene a la hora de eliminar registros profundizar el análisis por tipo de propiedad y barrio
- TODO: Calcular la media de las propiedades segun tipo de propiedad, cantidad de ambientes y barrio
- TODO: Ver que hacer con building_layout y orientation ya que hay pocos datos pero pueden ser variables interesantes
- TODO: En la parte de unificación de moneda, podemos armar un script para traer los distintos tipos de cambio y usar un promedio (Lo tengo hecho en otro lado). Ver opción de armar campo Fcha de publicción pra convertir expensa a valor dolar de ese momento
- TODO: Analizar la relación de expensa vs precio de venta
- TARGET: predecir precio de venta en usd
- TODO: ver vble antique, si la tomamos como categorica. Como tratar "en construccion" y "a estrenar". Cuanto impacta en el precio
- TODO: ELimnar aquellos registros que tienen 0 en valor de venta, ya que es nuestro target

# Lectura del Dataset

In [58]:
df = pd.read_csv("https://drive.google.com/uc?id=1ytSrLbhEWu4lm9ADZL7bUgBUeVOvywGL",sep = ";", on_bad_lines='skip')
df.sample(5)

,id,name,operation_type,operation_currency,operation_amount,expenses_currency,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,building_layout,orientation,number_of_floors,apartments_per_floor,real_estate_type,posting_type,publisher_id,publisher_name,address,city_id,city,state_id,state,country_id,country,latitude,longitude,reserved,publication_antiquity,url
695,48055647,Aer Julián Álvarez,En Pozo,USD,"122,000.00",NaN,NaN,45.00,35.00,1.00,1.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,Vertical,DEVELOPMENT,17020690,OCAMPO PROPIEDADES,Julián Álvarez 1253,V1-C-1003694,Palermo,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.5947677,-58.4266989,False,NaN,https://www.zonaprop.com.ar/propiedades/empren...
9894,53206823,Venta Departamento 3 Amb Belgrano Luminoso,Venta,USD,"110,000.00",ARS,"30,000.00",61.00,61.00,3.00,2.00,1.00,NaN,50,NaN,NaN,NaN,NaN,Apartamento,PROPERTY,30013644,RE/MAX POWER,Virrey del Pino 1500,V1-C-1003652,Belgrano,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.5609738,-58.442607,False,Publicado hace 72 días,https://www.zonaprop.com.ar/propiedades/clasif...
15716,53572240,"Excelente Torre de Peña, Ocampo y Figueroa Alc...",Venta,USD,"1,480,000.00",ARS,"96,540.00",231.00,217.00,5.00,3.00,2.00,2.00,55,Frente,NE,NaN,NaN,Apartamento,PROPERTY,17840367,GPS real estate,Ortiz de Ocampo al 2800,V1-C-1003694,Palermo,V1-B-6,Capital Federal,V1-A-1,Argentina,"-34,57871789","-58,40288575",False,Publicado hace 13 días,https://www.zonaprop.com.ar/propiedades/clasif...
12970,53431870,Studio Piso Altisimo en Venta en Quartier del ...,Venta,USD,"150,500.00",ARS,0.00,35.00,31.00,1.00,1.00,1.00,NaN,En construcción,NaN,NaN,NaN,NaN,Apartamento,PROPERTY,17032431,Unité Quartier,Quartier Del Bajo.,V1-C-1003655,Puerto Madero,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.6245423,-58.3759895,False,Publicado hace 35 días,https://www.zonaprop.com.ar/propiedades/clasif...
12728,53413299,Departamento 3 Amb Al Frente con Muy Bajas Exp...,Venta,USD,"69,900.00",ARS,"6,000.00",55.00,55.00,3.00,2.00,1.00,NaN,40,NaN,NaN,NaN,NaN,Apartamento,PROPERTY,30028597,Longo Moriello Propiedades,Joaquín V. González 595,V1-C-1003654,Floresta,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.6266907,-58.4816908,False,Publicado hace 37 días,https://www.zonaprop.com.ar/propiedades/clasif...


In [59]:
!pip install ydata_profiling

from ydata_profiling import ProfileReport


In [60]:
# Usando pandas
profilingreport = ProfileReport(df, title='dptos', minimal=True)
profilingreport


Output hidden; open in https://colab.research.google.com to view.

# Limpieza de Datos

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16945 entries, 0 to 16944
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     16945 non-null  int64  
 1   name                   16945 non-null  object 
 2   operation_type         16945 non-null  object 
 3   operation_currency     16912 non-null  object 
 4   operation_amount       16912 non-null  float64
 5   expenses_currency      14182 non-null  object 
 6   expenses_amount        14182 non-null  float64
 7   total_mts              16878 non-null  float64
 8   covered_mts            15883 non-null  float64
 9   rooms                  15304 non-null  float64
 10  bedrooms               13421 non-null  float64
 11  bathrooms              15470 non-null  float64
 12  garages                5066 non-null   float64
 13  antique                15461 non-null  object 
 14  building_layout        10121 non-null  object 
 15  or

## Eliminación de columnas

### Eliminación por imposibilidad de estimar

Existen algunas columnas que tienen muy pocos datos y decidimos eliminarlas ya que no hay forma objetiva de imputarles un valor. Entre ellas tenemos:
- number_of_floors
- apartments_per_floor

In [62]:
df.drop(columns=['number_of_floors','apartments_per_floor'], inplace=True)


In [63]:
df.head(1)

,id,name,operation_type,operation_currency,operation_amount,expenses_currency,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,building_layout,orientation,real_estate_type,posting_type,publisher_id,publisher_name,address,city_id,city,state_id,state,country_id,country,latitude,longitude,reserved,publication_antiquity,url
0,20374238,Excelente 3 Ambientes - Vista Plena a Plaza - ...,Venta,USD,"158,000.00",ARS,"40,000.00",55.00,55.00,3.00,2.00,1.00,NaN,50,Frente,NE,Apartamento,PROPERTY,17049913,Sanchez Naon,Mendoza 2628,V1-C-1003652,Belgrano,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.56195859172674,-58.45909752774658,False,Publicado hace más de 1 año,https://www.zonaprop.com.ar/propiedades/clasif...


### Eliminacion por referencia identica

Existen algunas columnas que referencian lo mismo ya que cuenta con el id (Interno de la Base de datos) y el nombre. De estas columnas nos quedaremos solo con el nombre. Entre ellas podemos encontrar a:
- publisher_id
- city_id
- state_id
- country_id

In [64]:
df.drop(columns=['publisher_id','city_id','state_id','country_id','publisher_name','name','address'], inplace=True)

In [65]:
df.head(1)

,id,operation_type,operation_currency,operation_amount,expenses_currency,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,building_layout,orientation,real_estate_type,posting_type,city,state,country,latitude,longitude,reserved,publication_antiquity,url
0,20374238,Venta,USD,"158,000.00",ARS,"40,000.00",55.00,55.00,3.00,2.00,1.00,NaN,50,Frente,NE,Apartamento,PROPERTY,Belgrano,Capital Federal,Argentina,-34.56195859172674,-58.45909752774658,False,Publicado hace más de 1 año,https://www.zonaprop.com.ar/propiedades/clasif...


### Eliminacion por no aportar al análisis

La columna de url es una columna que no nos aporta valor para los análisis que realizaremos ni tampoco para la predicción

In [66]:
df.drop(columns=['url'], inplace=True)

## Eliminación de valores no analizables

Eliminamos los registros correspondientes a Vertical, Hotel, Edificio, Horizontal y Fondo de Comercio ya que estos tipos de propiedad refieren a la venta de un edificio en su totalidad, varias unidades en simultaneo o negocios que no son el objeto de análisis

In [67]:
#df = df[~df['real_estate_type'].isin(['Vertical','Hotel','Edificio','Horizontal','Fondo de Comercio','Terrenos'])]
df = df[df['real_estate_type'].isin(['Apartamento','PH','Casa'])]


## Estimación de valores

### Reemplazar por cero

Existen valores que podemos reemplazar por 0 ya que la ausencia de los mismos podemos interpretar que refiere a la no existencia de los mismos.
Es una variable que el vendedor public, por lo que asumimos que los valores faltantes son de publicaciones "sin cochera" que no completan ese dato


In [68]:
df['garages'] = df['garages'].fillna(0)

<ipython-input-68-c64cd41b6c32>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['garages'] = df['garages'].fillna(0)


En el caso de bedrooms y rooms, el análisis requiere profundizar aún más ya que según el tipo de propiedad podemos asumir distintas cosas

Si rooms + bedrroms + bathrooms es NAN, elimino el valor por falta de info certera

In [69]:
mascara = df[['rooms','bedrooms','bathrooms']].isna().all(axis=1)

df = df[~mascara]


In [70]:
df

,id,operation_type,operation_currency,operation_amount,expenses_currency,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,building_layout,orientation,real_estate_type,posting_type,city,state,country,latitude,longitude,reserved,publication_antiquity
0,20374238,Venta,USD,"158,000.00",ARS,"40,000.00",55.00,55.00,3.00,2.00,1.00,0.00,50,Frente,NE,Apartamento,PROPERTY,Belgrano,Capital Federal,Argentina,-34.56195859172674,-58.45909752774658,False,Publicado hace más de 1 año
1,21023929,Venta,USD,"185,000.00",ARS,"87,000.00",84.00,84.00,4.00,3.00,1.00,0.00,50,NaN,NaN,Apartamento,PROPERTY,Recoleta,Capital Federal,Argentina,-34.59329994011797,-58.39768096983641,False,Publicado hace más de 1 año
2,22003010,Venta,USD,"695,000.00",ARS,"89,000.00",140.00,140.00,4.00,3.00,3.00,1.00,10,Frente,N,Apartamento,PROPERTY,Palermo,Capital Federal,Argentina,-34.5748601,-58.4221215,False,Publicado hace más de 1 año
4,24339396,Venta,USD,"120,000.00",ARS,0.00,100.00,70.00,3.00,2.00,1.00,0.00,12,Frente,NaN,PH,PROPERTY,Parque Chacabuco,Capital Federal,Argentina,-34.64087103351533,-58.42717985291745,False,Publicado hace más de 1 año
6,28015090,Venta,USD,"2,100,000.00",ARS,"200,000.00",211.00,211.00,5.00,3.00,3.00,2.00,5,Frente,N,Apartamento,PROPERTY,Palermo,Capital Federal,Argentina,-34.5765491,-58.40487150000001,False,Publicado hace más de 1 año
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16940,53622858,Venta,USD,"345,000.00",ARS,"145,000.00",192.00,129.00,5.00,4.00,3.00,1.00,20,Contrafrente,NE,Apartamento,PROPERTY,Caballito,Capital Federal,Argentina,"-34,61905815","-58,45986718",False,Publicado hace 5 días
16941,53622861,Venta,USD,"345,000.00",ARS,"145,000.00",192.00,129.00,5.00,4.00,3.00,1.00,20,NaN,NE,Casa,PROPERTY,Flores,Capital Federal,Argentina,"-34,61905815","-58,45986718",False,Publicado hace 5 días
16942,53623222,Venta,USD,"275,000.00",ARS,"126,000.00",179.00,90.00,4.00,3.00,2.00,0.00,40,NaN,SE,Apartamento,PROPERTY,Belgrano,Capital Federal,Argentina,"-34,5671024","-58,4521356",False,Publicado hace 5 días
16943,53623267,Venta,USD,0.00,ARS,0.00,450.00,450.00,4.00,3.00,4.00,1.00,A estrenar,NaN,NE,Casa,PROPERTY,Belgrano,Capital Federal,Argentina,"-34,5685002","-58,4402606",False,Publicado hace 5 días


Pruebo imputacion por KNN

In [71]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
imputed_array = imputer.fit_transform(df[['rooms','bedrooms','bathrooms']])
df[['rooms','bedrooms','bathrooms']] = imputed_array

In [72]:
df[df['bedrooms'].isna()].real_estate_type.value_counts()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13750 entries, 0 to 16944
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     13750 non-null  int64  
 1   operation_type         13750 non-null  object 
 2   operation_currency     13750 non-null  object 
 3   operation_amount       13750 non-null  float64
 4   expenses_currency      12401 non-null  object 
 5   expenses_amount        12401 non-null  float64
 6   total_mts              13742 non-null  float64
 7   covered_mts            13333 non-null  float64
 8   rooms                  13750 non-null  float64
 9   bedrooms               13750 non-null  float64
 10  bathrooms              13750 non-null  float64
 11  garages                13750 non-null  float64
 12  antique                13521 non-null  object 
 13  building_layout        10115 non-null  object 
 14  orientation            6999 non-null   object 
 15  real_es

En el caso de covered_mts, al igual que bedrooms y rooms, el análisis requiere profundizar aún más ya que según el tipo de propiedad podemos asumir distintas cosas

In [73]:
df[df['covered_mts'].isna()].real_estate_type.value_counts()

real_estate_type
Apartamento    362
Casa            29
PH              26
Name: count, dtype: int64

Pruebo imputacion de covered mts por la media

In [74]:
media_covered = df['covered_mts'].mean()

df['covered_mts'] = df['covered_mts'].fillna(media_covered)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13750 entries, 0 to 16944
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     13750 non-null  int64  
 1   operation_type         13750 non-null  object 
 2   operation_currency     13750 non-null  object 
 3   operation_amount       13750 non-null  float64
 4   expenses_currency      12401 non-null  object 
 5   expenses_amount        12401 non-null  float64
 6   total_mts              13742 non-null  float64
 7   covered_mts            13750 non-null  float64
 8   rooms                  13750 non-null  float64
 9   bedrooms               13750 non-null  float64
 10  bathrooms              13750 non-null  float64
 11  garages                13750 non-null  float64
 12  antique                13521 non-null  object 
 13  building_layout        10115 non-null  object 
 14  orientation            6999 non-null   object 
 15  real_es

### Reemplazar por máximo posible

Para los registros que tengan un valor mayor de metros cubiertos comparado con metros totales, le seteamos como máximo el de metros totales

In [75]:
condition = df['covered_mts'] > df['total_mts']
df.loc[condition, 'covered_mts'] = df.loc[condition, 'total_mts']

In [76]:
condition = df['total_mts'].isna()
df.loc[condition, 'total_mts'] = df.loc[condition, 'covered_mts']

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13750 entries, 0 to 16944
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     13750 non-null  int64  
 1   operation_type         13750 non-null  object 
 2   operation_currency     13750 non-null  object 
 3   operation_amount       13750 non-null  float64
 4   expenses_currency      12401 non-null  object 
 5   expenses_amount        12401 non-null  float64
 6   total_mts              13750 non-null  float64
 7   covered_mts            13750 non-null  float64
 8   rooms                  13750 non-null  float64
 9   bedrooms               13750 non-null  float64
 10  bathrooms              13750 non-null  float64
 11  garages                13750 non-null  float64
 12  antique                13521 non-null  object 
 13  building_layout        10115 non-null  object 
 14  orientation            6999 non-null   object 
 15  real_es

## Unificación de moneda

Existen registros que estan en moneda ARS ($). Estos los transformaremos a USD utilizando el TC seleccionado para asi unificar los análisis

In [77]:
dolar_bna = 909.5
dolar_blue = 1220
currency_rate = (dolar_bna + dolar_blue) / 2
df['operation_amount'] = np.where(df['operation_currency'] == '$', df['operation_amount'] / currency_rate, df['operation_amount'])
df['operation_currency'] = 'USD'
df['expenses_amount'] = np.where(df['expenses_currency'] == 'ARS', df['expenses_amount'] / currency_rate, df['expenses_amount'])
df['expenses_currency'] = 'USD'

## Transformación de columnas string

La columna antique no solo contiene registros con antiguedad en años, sino que tambien contamos con valores como por ejemplo "A estrenar" o "En construcción". Estos los reemplazaremos por 0 y transformaremos la columna en flotante (Podria ser por entero pero como tiene NaN, pandas no lo soporta)

In [78]:
df.antique.value_counts()

antique
A estrenar         3721
En construcción    1656
40                  948
50                  910
45                  458
                   ... 
134                   1
1930                  1
138                   1
99                    1
129                   1
Name: count, Length: 124, dtype: int64

In [79]:
df['antique'] = df['antique'].replace('A estrenar', 0)
df['antique'] = df['antique'].replace('En construcción', -1)

df['antique'] = df['antique'].astype(float)

In [80]:
#Prueba completo los Nan de antiguedad con valores muy grandes

## Eliminación de Outliers

In [81]:
df.describe()

,id,operation_amount,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique
count,"13,750.00","13,750.00","12,401.00","13,750.00","13,750.00","13,750.00","13,750.00","13,750.00","13,750.00","13,521.00"
mean,"52,504,865.50","294,948.84",65.49,116.04,96.38,3.10,2.11,1.69,0.46,22.57
std,"1,691,064.20","415,037.63",214.51,121.16,90.67,1.53,1.12,0.97,1.10,48.23
min,"20,374,238.00",0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.00,-1.00
25%,"52,266,741.75","113,000.00",0.00,52.00,46.00,2.00,1.00,1.00,0.00,0.00
50%,"53,115,942.50","176,700.00",20.66,81.00,70.00,3.00,2.00,1.00,0.00,10.00
75%,"53,433,061.75","319,000.00",75.14,139.00,112.00,4.00,3.00,2.00,1.00,40.00
max,"53,623,344.00","12,000,000.00","17,822.68","5,878.00","3,367.00",32.00,18.00,20.00,72.00,"2,024.00"


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13750 entries, 0 to 16944
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     13750 non-null  int64  
 1   operation_type         13750 non-null  object 
 2   operation_currency     13750 non-null  object 
 3   operation_amount       13750 non-null  float64
 4   expenses_currency      13750 non-null  object 
 5   expenses_amount        12401 non-null  float64
 6   total_mts              13750 non-null  float64
 7   covered_mts            13750 non-null  float64
 8   rooms                  13750 non-null  float64
 9   bedrooms               13750 non-null  float64
 10  bathrooms              13750 non-null  float64
 11  garages                13750 non-null  float64
 12  antique                13521 non-null  float64
 13  building_layout        10115 non-null  object 
 14  orientation            6999 non-null   object 
 15  real_es

Eliminaremos aquellos outliers que distorcionan el dataset. Para ello, aplicaremos como regla general eliminar los registros que esten sobre el percentil 0.999. Esto lo haremos para las columnas:
- operation_amount
- expenses_amount
- total_mts
- covered_mts
- rooms
- bedrooms
- bathrooms
- garages

In [83]:
# Lista de columnas que queremos filtrar
columns_to_filter = ['operation_amount', 'expenses_amount', 'total_mts', 'covered_mts', 'rooms', 'bedrooms', 'bathrooms', 'garages', 'antique']

# Calcular el percentil para cada columna en la lista
percentile = df[columns_to_filter].quantile(0.999)

# Filtrar el DataFrame para eliminar valores por encima del percentil en las columnas seleccionadas
for col in columns_to_filter:
    df = df[df[col] <= percentile[col]]

In [84]:
df.describe()

,id,operation_amount,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique
count,"12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,252.00"
mean,"52,597,277.25","289,793.86",61.38,111.08,93.10,3.07,2.09,1.65,0.43,21.80
std,"1,556,842.24","355,162.43",112.60,91.52,74.94,1.43,1.05,0.90,0.71,24.92
min,"20,374,238.00",0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.00,-1.00
25%,"52,342,580.25","114,187.50",0.00,53.00,46.00,2.00,1.00,1.00,0.00,0.00
50%,"53,148,917.00","178,000.00",20.66,80.00,70.00,3.00,2.00,1.00,0.00,10.00
75%,"53,443,952.25","320,000.00",75.14,135.00,110.00,4.00,3.00,2.00,1.00,41.00
max,"53,623,344.00","4,500,000.00","1,226.83","1,000.00",859.00,12.00,8.00,7.00,10.00,124.00


In [85]:
# Lista de columnas que queremos filtrar
columns_to_filter = ['operation_amount', 'expenses_amount', 'total_mts', 'covered_mts', 'rooms', 'bedrooms', 'bathrooms', 'garages', 'antique']

# Calcular el percentil para cada columna en la lista
percentile = df[columns_to_filter].quantile(0.001)

# Filtrar el DataFrame para eliminar valores por encima del percentil en las columnas seleccionadas
for col in columns_to_filter:
    df = df[df[col] >= percentile[col]]

In [86]:
df.describe()

,id,operation_amount,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique
count,"12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,233.00"
mean,"52,597,560.75","290,073.60",61.40,111.18,93.22,3.07,2.09,1.65,0.43,21.78
std,"1,555,539.20","355,344.87",112.67,91.53,74.93,1.43,1.05,0.90,0.71,24.92
min,"20,374,238.00",0.00,0.00,21.00,18.00,1.00,1.00,1.00,0.00,-1.00
25%,"52,342,257.00","114,900.00",0.00,53.00,46.00,2.00,1.00,1.00,0.00,0.00
50%,"53,148,317.00","178,000.00",20.66,80.00,70.00,3.00,2.00,1.00,0.00,10.00
75%,"53,443,953.00","320,000.00",75.14,135.00,110.00,4.00,3.00,2.00,1.00,41.00
max,"53,623,344.00","4,500,000.00","1,226.83","1,000.00",859.00,12.00,8.00,7.00,10.00,124.00


## Eliminación de valores faltantes

Eliminamos los registros que no tienen latitud, longitud y antique ya que estos son muy pocos y la perdida no presenta un problema

In [87]:
df.dropna(subset=['latitude','longitude','antique'], inplace=True)

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12232 entries, 0 to 16944
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     12232 non-null  int64  
 1   operation_type         12232 non-null  object 
 2   operation_currency     12232 non-null  object 
 3   operation_amount       12232 non-null  float64
 4   expenses_currency      12232 non-null  object 
 5   expenses_amount        12232 non-null  float64
 6   total_mts              12232 non-null  float64
 7   covered_mts            12232 non-null  float64
 8   rooms                  12232 non-null  float64
 9   bedrooms               12232 non-null  float64
 10  bathrooms              12232 non-null  float64
 11  garages                12232 non-null  float64
 12  antique                12232 non-null  float64
 13  building_layout        9189 non-null   object 
 14  orientation            6249 non-null   object 
 15  real_es

# Modelado

## Selección de columnas para el modelo

In [89]:
df_model = df[['operation_type','expenses_amount','total_mts','covered_mts','rooms','bedrooms','bathrooms','garages','real_estate_type','posting_type','city','antique','operation_amount']]
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12232 entries, 0 to 16944
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   operation_type    12232 non-null  object 
 1   expenses_amount   12232 non-null  float64
 2   total_mts         12232 non-null  float64
 3   covered_mts       12232 non-null  float64
 4   rooms             12232 non-null  float64
 5   bedrooms          12232 non-null  float64
 6   bathrooms         12232 non-null  float64
 7   garages           12232 non-null  float64
 8   real_estate_type  12232 non-null  object 
 9   posting_type      12232 non-null  object 
 10  city              12232 non-null  object 
 11  antique           12232 non-null  float64
 12  operation_amount  12232 non-null  float64
dtypes: float64(9), object(4)
memory usage: 1.3+ MB


## Generación de dummies

In [90]:
#df_model = pd.get_dummies(df_model, columns=['operation_type'], prefix='optype')
df_model = pd.get_dummies(df_model, columns=['real_estate_type'], prefix='restype')
df_model = pd.get_dummies(df_model, columns=['posting_type'], prefix='postype')
df_model = pd.get_dummies(df_model, columns=['city'], prefix='city')

In [91]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12232 entries, 0 to 16944
Data columns (total 71 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   operation_type             12232 non-null  object 
 1   expenses_amount            12232 non-null  float64
 2   total_mts                  12232 non-null  float64
 3   covered_mts                12232 non-null  float64
 4   rooms                      12232 non-null  float64
 5   bedrooms                   12232 non-null  float64
 6   bathrooms                  12232 non-null  float64
 7   garages                    12232 non-null  float64
 8   antique                    12232 non-null  float64
 9   operation_amount           12232 non-null  float64
 10  restype_Apartamento        12232 non-null  bool   
 11  restype_Casa               12232 non-null  bool   
 12  restype_PH                 12232 non-null  bool   
 13  postype_PROPERTY           12232 non-null  bool   


## Armado del modelo

### Generacion de X e y

In [92]:
X = df_model.drop(columns=['operation_amount','operation_type'])
y = df_model['operation_amount']

In [93]:
df.city.value_counts()

city
Palermo                 2116
Belgrano                1595
Caballito               1054
Recoleta                 901
Núñez                    561
Villa Urquiza            548
Villa Crespo             442
Almagro                  400
Flores                   377
Puerto Madero            327
Saavedra                 301
Colegiales               292
Barrio Norte             265
Villa Devoto             244
Balvanera                224
Villa del Parque         204
Retiro                   160
Coghlan                  150
San Telmo                135
Barracas                 127
Monserrat                123
San Cristobal            119
Boedo                    114
Villa Pueyrredón         104
Villa Luro               101
Floresta                  93
Parque Chacabuco          92
La Paternal               90
Liniers                   85
Chacarita                 82
Monte Castro              76
Congreso                  67
Villa Ortuzar             63
Parque Patricios          61
San Nicol

### Train-Test Split

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=50)

In [95]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
X_val = sc_X.transform(X_val)

# Heurística Base

In [96]:
class BaseHeuristic():
    def __init__(self):
        self.mean = 0

    def fit(self, X, Y):
        self.mean = np.mean(Y)

    def predict(self, X):
        return pd.Series([self.mean for x in X],name="operation_amount")

In [97]:
# Crear el modelo Heurístico
model = BaseHeuristic()

# Entrenar el modelo
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [98]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
mpe = mean_porcentual_error(y_test, y_pred)

print("R-cuadrado en test:", r2)
print("Error absoluto medio:", mae)
print("Error cuadratico medio:", mse)
print("Raiz de error cuadratico medio:", rmse)
print(f"Error absoluto porcentual medio: {mape*100:.2f}%")
print(f"Error porcentual medio: {mpe*100:.2f}%")

R-cuadrado en test: -0.00013526169600264204
Error absoluto medio: 210274.74382596518
Error cuadratico medio: 139805721116.48953
Raiz de error cuadratico medio: 373906.03246870666
Error absoluto porcentual medio: 1014089716944241819648.00%
Error porcentual medio: -inf%


### Regresión Lineal

In [99]:
# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

LinearRegression()

In [100]:
# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [101]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
mpe = mean_porcentual_error(y_test, y_pred)

print("R-cuadrado en test:", r2)
print("Error absoluto medio:", mae)
print("Error cuadratico medio:", mse)
print("Raiz de error cuadratico medio:", rmse)
print(f"Error absoluto porcentual medio: {mape*100:.2f}%")
print(f"Error porcentual medio: {mpe*100:.2f}%")

R-cuadrado en test: 0.6562607416473962
Error absoluto medio: 99931.0543128949
Error cuadratico medio: 48050215536.38634
Raiz de error cuadratico medio: 219203.59380353766
Error absoluto porcentual medio: 2551153894027536367616.00%
Error porcentual medio: -inf%


In [102]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge_dptos = Ridge()

grid = GridSearchCV(ridge_dptos,
                    {"alpha": np.linspace(0, 20, 1000)},
                    refit=True,
                    cv=5,
                    scoring='neg_mean_absolute_error')
grid.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': array([ 0.        ,  0.02002002,  0.04004004,  0.06006006,  0.08008008,
        0.1001001 ,  0.12012012,  0.14014014,  0.16016016,  0.18018018,
        0.2002002 ,  0.22022022,  0.24024024,  0.26026026,  0.28028028,
        0.3003003 ,  0.32032032,  0.34034034,  0.36036036,  0.38038038,
        0.4004004 ,  0.42042042,  0.44044044,  0.46046046,  0.48048048,
        0.5005005 ,  0.52052052,  0.5...
       19.41941942, 19.43943944, 19.45945946, 19.47947948, 19.4994995 ,
       19.51951952, 19.53953954, 19.55955956, 19.57957958, 19.5995996 ,
       19.61961962, 19.63963964, 19.65965966, 19.67967968, 19.6996997 ,
       19.71971972, 19.73973974, 19.75975976, 19.77977978, 19.7997998 ,
       19.81981982, 19.83983984, 19.85985986, 19.87987988, 19.8998999 ,
       19.91991992, 19.93993994, 19.95995996, 19.97997998, 20.        ])},
             scoring='neg_mean_absolute_error')

In [103]:
from sklearn.metrics import mean_absolute_error

ridge_dptos_best = grid.best_estimator_

y_pred = ridge_dptos_best.predict(X_test)


In [104]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
mpe = mean_porcentual_error(y_test, y_pred)

print("R-cuadrado en test:", r2)
print("Error absoluto medio:", mae)
print("Error cuadratico medio:", mse)
print("Raiz de error cuadratico medio:", rmse)
print(f"Error absoluto porcentual medio: {mape*100:.2f}%")
print(f"Error porcentual medio: {mpe*100:.2f}%")

R-cuadrado en test: 0.656017957075969
Error absoluto medio: 99905.57798126707
Error cuadratico medio: 48084153617.94819
Raiz de error cuadratico medio: 219280.99237724228
Error absoluto porcentual medio: 2547600974861357285376.00%
Error porcentual medio: -inf%


# Optimización con Optuna



In [105]:
!pip install optuna
import optuna
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error

In [ ]:
# Función objetivo para la optimización de Optuna
def objective(trial):
    # Definir los hiperparámetros a optimizar
    alpha = trial.suggest_loguniform('alpha', 0.001, 1000.0)

    # Crear y entrenar el modelo de regresión Ridge con los hiperparámetros sugeridos
    model = Ridge(alpha=alpha)
    model.fit(X_train_selected, y_train)

    # Calcular la métrica de evaluación (por ejemplo, el error cuadrático medio)
    y_pred = model.predict(X_val_selected)
    mae = mean_absolute_error(y_val, y_pred)

    return mae

# Iterar sobre diferentes números de características para la eliminación recursiva de características
best_mae = float('inf')
best_features = None

for num_features in range(1, len(X.columns) + 1):
    # Realizar la eliminación recursiva de características
    selector = RFE(estimator=Ridge(), n_features_to_select=num_features)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_val_selected = selector.transform(X_val)

    # Crear un estudio de Optuna y ejecutar la optimización
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=1000)

    # Obtener el error cuadrático medio del modelo para las características seleccionadas
    if study.best_trial.value < best_mae:
        best_mae = study.best_trial.value
        best_features = selector.support_



Streaming output truncated to the last 5000 lines.
[I 2024-06-05 14:15:53,636] Trial 333 finished with value: 98987.19404709035 and parameters: {'alpha': 287.2278012987184}. Best is trial 308 with value: 98437.35648828605.
<ipython-input-106-3cd220ec141f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 0.001, 1000.0)
[I 2024-06-05 14:15:53,713] Trial 334 finished with value: 98585.30936430323 and parameters: {'alpha': 772.2130256248032}. Best is trial 308 with value: 98437.35648828605.
<ipython-input-106-3cd220ec141f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 0.001, 1000.0)
[I 20

In [ ]:
best_features

array([ True,  True,  True, False, False,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False,  True, False, False, False, False, False,
       False,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [ ]:
# Utilizar las mejores características encontradas para entrenar el modelo final
X_train_final = pd.DataFrame(X_train).loc[:, best_features]



In [ ]:
X_test_final = pd.DataFrame(X_test).loc[:, best_features]


In [ ]:
final_model = Ridge()
final_model.fit(X_train_final, y_train)  # Aquí se cambia y_train por y_val



Ridge()

In [ ]:
# Hacer predicciones y evaluar el modelo final
y_pred_final = final_model.predict(X_test_final)  # Aquí se cambia X_test por X_val_final




In [ ]:
r2 = r2_score(y_test, y_pred_final)
mae = mean_absolute_error(y_test, y_pred_final)
mse = mean_squared_error(y_test, y_pred_final)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred_final)
mpe = mean_porcentual_error(y_test, y_pred_final)

print("R-cuadrado en test:", r2)
print("Error absoluto medio:", mae)
print("Error cuadratico medio:", mse)
print("Raiz de error cuadratico medio:", rmse)
print(f"Error absoluto porcentual medio: {mape*100:.2f}%")
print(f"Error porcentual medio: {mpe*100:.2f}%")

R-cuadrado en test: 0.6548544146007611
Error absoluto medio: 100277.34687793131
Error cuadratico medio: 48246801512.71419
Raiz de error cuadratico medio: 219651.54566429573
Error absoluto porcentual medio: 2564231928520927870976.00%
Error porcentual medio: -inf%
